In [1]:
# Importing data paths
import sys
sys.path.insert(1, '/Users/piyush/Desktop/dsml_Portfolio/podcast/final_push')
from config import PODCASTS_DATABASE_PATH_RAW, USER_REVIEWS_DATABASE_PATH_RAW, PODCASTS_DATABASE_PATH_PROCESSED, USER_REVIEWS_DATABASE_PATH_PROCESSED
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
%matplotlib inline
from IPython.display import Image

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/piyush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/piyush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/piyush/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/piyush/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
PODCASTS_DATABASE_PATH_RAW, USER_REVIEWS_DATABASE_PATH_RAW

('/Users/piyush/Desktop/dsml_Portfolio/podcast/final_push/data/raw/podcasts.csv',
 '/Users/piyush/Desktop/dsml_Portfolio/podcast/final_push/data/raw/user_reviews.csv')

In [4]:
# Loading data 
df_podcasts = pd.read_csv(PODCASTS_DATABASE_PATH_RAW,header=None)
df_users = pd.read_csv(USER_REVIEWS_DATABASE_PATH_RAW,header=None)

In [11]:
columns_pod = ['id','name','url','studio','category','episode_count','avg_rating','total_ratings','description']
columns_users = ['id','podcasts_id','username','review_title','review','rating','date']

df_podcasts.columns = columns_pod
df_users.columns = columns_users
df_users = df_users.drop('id', axis=1)

In [ ]:
df_users['review']

In [6]:
pip install vaderSentiment


  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer()
text_1 = tags_list[0]
text_2 =  tags_list[1]
sent_1 = sentiment.polarity_scores(text_1)
sent_2 = sentiment.polarity_scores(text_2)
print("Sentiment of text 1:", sent_1)
print("Sentiment of text 2:", sent_2)

Sentiment of text 1: {'neg': 0.023, 'neu': 0.735, 'pos': 0.243, 'compound': 0.9611}
Sentiment of text 2: {'neg': 0.2, 'neu': 0.717, 'pos': 0.084, 'compound': -0.8669}


In [19]:
pop = []
for i in tags_list[0:1000]:
    pop.append(sentiment.polarity_scores(i))

In [21]:
pop[67]

{'neg': 0.257, 'neu': 0.514, 'pos': 0.229, 'compound': 0.128}

In [22]:
tags_list[67]

'Outstanding piece.  Today’s political climate compared with historical happenings.   Serious and scary.'

In [25]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('The food was great!')
classifier.predict(sentence)

# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

2022-12-28 16:32:04,335 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /var/folders/hk/dkqk41797ls590n4w_sfpy180000gn/T/tmpo93b0ouw


100%|██████████| 265512723/265512723 [01:29<00:00, 2980304.30B/s]

2022-12-28 16:33:33,675 copying /var/folders/hk/dkqk41797ls590n4w_sfpy180000gn/T/tmpo93b0ouw to cache at /Users/piyush/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-12-28 16:33:33,938 removing temp file /var/folders/hk/dkqk41797ls590n4w_sfpy180000gn/T/tmpo93b0ouw
2022-12-28 16:33:34,144 loading file /Users/piyush/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Sentence above is:  ['Sentence: "The food was great !"'/'POSITIVE' (0.9961)]


In [30]:
sentence = Sentence(tags_list[0])
classifier.predict(sentence)

# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

Sentence above is:  ['Sentence: "It ’s so amazing that this is deja vu . The story is so well narrated by Rachel as usual , and it just makes you want more . It also surprises me to hear that Make America Great has surfaced again in the 21st century and the title is used just has a sham as it was used in the 1940 ’s . Rachel , you ALWAYS do such great work . Well done presentation , I ’m anxious to hear more !! !"'/'POSITIVE' (0.9994)]


In [36]:
for i in tags_list[0:100]:
    sentence = Sentence(i)
    classifier.predict(sentence)

# print sentence with predicted labels
    print(sentence.labels)

['Sentence: "It ’s so amazing that this is deja vu . The story is so well narrated by Rachel as usual , and it just makes you want more . It also surprises me to hear that Make America Great has surfaced again in the 21st century and the title is used just has a sham as it was used in the 1940 ’s . Rachel , you ALWAYS do such great work . Well done presentation , I ’m anxious to hear more !! !"'/'POSITIVE' (0.9994)]
['Sentence: "What a dizzying ride into the past as well as a terrifying mirror of the present . Listening to these riveting , granular details that we have come to expect and love from Rachel Maddow about this cancer that lived among us is horrifying and sobering at the same time . How can history seem so very now ? Cannot wait to learn more …! !"'/'POSITIVE' (0.9998)]
['Sentence: "I say this has happened and people look at me as if I am crazy ! This podcast is done exceptionally well !"'/'POSITIVE' (0.9965)]
['Sentence: "Crazy how history is repeating itself - just change 

In [40]:
sentence.labels

['Sentence: "Just , WOW !! !"'/'POSITIVE' (0.9968)]

In [27]:
sentence

Sentence: "The food was great !" → POSITIVE (0.9961)

In [1]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
#data = ["I love you", "I hate you"]
#sentiment_pipeline(data)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [2]:
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [12]:
tags_list = list(df_users['review'].values)

In [45]:
results = sentiment_pipeline(tags_list[0:100])

In [40]:
res_sub = results[0:10]

In [44]:
res_sub[0]['label']

'POSITIVE'

In [38]:
542285/100 

5422.85